# Example where block/coordinate descent doesn't reach the same solution

They should converge to the same solution, but in practice they don't, and this notebook illustrates such an example using a $10 \times 10$ matrix. 

In [9]:
# load packages needed for this tutorial
using Revise
using Knockoffs
using Random
using GLMNet
using Distributions
using LinearAlgebra
using ToeplitzMatrices
using StatsBase
using CSV, DataFrames
using Plots
gr(fmt=:png);

function get_sigma(option::Int, p::Int)
    # note: groups are defined empirically within each simuation
    datadir = "/Users/biona001/Benjamin_Folder/research/4th_project_PRS/group_knockoff_test_data"
    if option == 1
        ρ = 0.7
        Σ = SymmetricToeplitz(ρ.^(0:(p-1))) |> Matrix
    elseif option == 2
        ρ = 0.7
        γ = 0.1
        groups = repeat(1:Int(p/5), inner=5)
        Σ = simulate_block_covariance(groups, ρ, γ)
    elseif option == 3
        covfile = CSV.read(joinpath(datadir, "CorG_2_127374341_128034347.txt"), DataFrame) # 3782 SNPs
        Σ = covfile |> Matrix{Float64}
        Σ = 0.99Σ + 0.01I #ensure PSD
    elseif option == 4
        df = CSV.read(joinpath(datadir, "21_37870779_38711704.csv"), DataFrame)
        Σ = df[:, 7:end] |> Matrix |> Symmetric |> Matrix
    elseif option == 5
        df = CSV.read(joinpath(datadir, "22_17674295_18295575.csv"), DataFrame)
        Σ = df[:, 7:end] |> Matrix |> Symmetric |> Matrix
    else
        error("Option should be 1-5 but was $option")
    end
    return Σ[1:p, 1:p]
end

sigma_option = 4
p = 10
Σ = get_sigma(sigma_option, p)

10×10 Matrix{Float64}:
  1.0         -0.00459261  -0.050176   …  -0.0104984   -0.0109347
 -0.00459261   1.0          0.0130137     -0.0208177   -0.0555964
 -0.050176     0.0130137    1.0            0.127436     0.0830088
 -0.00293355  -0.0237382    0.101152       0.280738     0.393509
 -0.0139809   -0.0341187    0.0287985      0.246218     0.447175
  0.0130641   -0.0132816   -0.100574   …  -0.00679832   0.00794679
  0.0152331   -0.0136096    0.154559      -0.182851    -0.237117
  0.0125588   -0.00528951   0.0383551     -0.187338    -0.119704
 -0.0104984   -0.0208177    0.127436       1.0          0.157373
 -0.0109347   -0.0555964    0.0830088      0.157373     1.0

$\Sigma$ is 10 by 10 with obvious blocks. Lets define 2 groups that capture this structure.

In [10]:
groups = repeat(1:2, inner=5)
groups

10-element Vector{Int64}:
 1
 1
 1
 1
 1
 2
 2
 2
 2
 2

Lets solve for SDP knockoff using various methods

In [11]:
@time equi, _, _ = solve_s_group(Symmetric(Σ), groups, :equi)
@time sdp_subopt, _, _ = solve_s_group(Symmetric(Σ), groups, :sdp_subopt)
@time sdp_block, _, _ = solve_s_group(Symmetric(Σ), groups, :sdp_block)
@time sdp_ccd, _, _ = solve_s_group(Symmetric(Σ), groups, :sdp, robust=false);

  0.000686 seconds (124 allocations: 64.344 KiB)
  0.016523 seconds (11.76 k allocations: 677.266 KiB)


┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/qzdJ6/src/Solvers/steppers/combined.jl:111
┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/qzdJ6/src/Solvers/steppers/combined.jl:111


  0.734290 seconds (1.50 M allocations: 76.498 MiB, 49.31% compilation time)


┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/qzdJ6/src/Solvers/steppers/combined.jl:111


  0.001420 seconds (154 allocations: 71.906 KiB)


In [12]:
equi

10×10 Matrix{Float64}:
  0.423534    -0.00194513  -0.0212512   …   0.0          0.0
 -0.00194513   0.423534     0.00551174      0.0          0.0
 -0.0212512    0.00551174   0.423534        0.0          0.0
 -0.00124246  -0.0100539    0.0428412       0.0          0.0
 -0.0059214   -0.0144504    0.0121971       0.0          0.0
  0.0          0.0          0.0         …  -0.00287932   0.00336574
  0.0          0.0          0.0            -0.0774437   -0.100427
  0.0          0.0          0.0            -0.0793442   -0.0506986
  0.0          0.0          0.0             0.423534     0.0666527
  0.0          0.0          0.0             0.0666527    0.423534

In [13]:
sdp_subopt

10×10 Matrix{Float64}:
  0.423534    -0.00194513  -0.0212512   …   0.0          0.0
 -0.00194513   0.423534     0.00551174      0.0          0.0
 -0.0212512    0.00551174   0.423534        0.0          0.0
 -0.00124246  -0.0100539    0.0428412       0.0          0.0
 -0.0059214   -0.0144504    0.0121971       0.0          0.0
  0.0          0.0          0.0         …  -0.00287932   0.00336574
  0.0          0.0          0.0            -0.0774437   -0.100427
  0.0          0.0          0.0            -0.0793442   -0.0506986
  0.0          0.0          0.0             0.423534     0.0666527
  0.0          0.0          0.0             0.0666527    0.423534

In [14]:
sdp_ccd

10×10 Matrix{Float64}:
  1.0         -0.00459261  -0.050176    …   0.0          0.0
 -0.00459261   1.0          0.00551642      0.0          0.0
 -0.050176     0.00551642   0.472893        0.0          0.0
 -0.00293355  -0.0237382    0.09563         0.0          0.0
 -0.0113535   -0.0341205    0.0155703       0.0          0.0
  0.0          0.0          0.0         …  -0.00679832   0.00794615
  0.0          0.0          0.0            -0.0790623   -0.100427
  0.0          0.0          0.0            -0.187338    -0.119704
  0.0          0.0          0.0             0.423534     0.123014
  0.0          0.0          0.0             0.123014     0.423532

In [15]:
sdp_block

10×10 Matrix{Float64}:
  1.0         -0.00459261  -0.050176   …   0.0          0.0
 -0.00459261   1.0          0.0130137      0.0          0.0
 -0.050176     0.0130137    1.0            0.0          0.0
 -0.00293355  -0.0237382    0.101152       0.0          0.0
 -0.0139809   -0.0341187    0.0287985      0.0          0.0
  0.0          0.0          0.0        …  -0.00679832   0.00794679
  0.0          0.0          0.0           -0.182851     0.103054
  0.0          0.0          0.0           -0.187338    -0.0698936
  0.0          0.0          0.0            1.0          0.157373
  0.0          0.0          0.0            0.157373     0.707497

Clearly, all approach reach the same solution except CCD. Lets check objective

In [16]:
m = 1
@show group_block_objective(Σ, equi, m, :equi)
@show group_block_objective(Σ, sdp_subopt, m, :sdp_subopt)
@show group_block_objective(Σ, sdp_ccd, m, :sdp)
@show group_block_objective(Σ, sdp_block, m, :sdp_block);

group_block_objective(Σ, equi, m, :equi) = 18.070493154727508
group_block_objective(Σ, sdp_subopt, m, :sdp_subopt) = 18.070492991251353
group_block_objective(Σ, sdp_ccd, m, :sdp) = 16.04998543594285
group_block_objective(Σ, sdp_block, m, :sdp_block) = 13.412249605360358
